In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkStreamWordCount")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/05/24 11:35:33 WARN Utils: Your hostname, Hocines-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.164 instead (on interface en0)
22/05/24 11:35:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/24 11:35:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Structured Streaming 
# based on Data Frame/ internally there is RDD
# Micro Batch
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, represent data comes from TCP/IP socket
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark shall connect the server running on port 9999
# then it will wait the input from server
# user/developer can write something on server console and Press Enter key
# to send to spark / client

22/05/24 11:35:36 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
lineStreamDf.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
from pyspark.sql.functions import split, explode, col
# split , split the string using space, convert line into word list
# explode project every element in list as element in stream [similar to flatMap]
# output is a stream of word
# Input "welcome to Kafka"
# split => ["welcome", "to", "Kafka"]
# explode makes every element as stream element
# welcome
# to
# Kafka
wordsDf = lineStreamDf.select(explode(split(lineStreamDf.value, " ")).alias("word"))
wordsDf.printSchema()

root
 |-- word: string (nullable = true)



In [6]:
# word count by group
# word  count
# Kafka  2..
wordCountDf = wordsDf.groupBy("word").count()

In [8]:
# print out on console 
# default batch, as soon possible , latest spark says 100 ms, expect 1 second delay
# check this cell or next cell
echoOnConsole = wordCountDf.writeStream.outputMode("complete").format("console").start()

22/05/24 11:39:02 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/y4/h7s1_2k959z8nfw5xqv11nmr0000gn/T/temporary-8dd12000-a144-460d-a683-2e5283a3e1bf. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/24 11:39:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|kafka|    3|
+-----+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
| spark|    3|
|hocine|    2|
+------+-----+



In [7]:
# print out on console 
# Output: update
# only print if there is difference in output, not printing all results
# default batch, as soon possible , latest spark says 100 ms, expect 1 second delay
echoOnConsole = wordCountDf.writeStream.outputMode("update").format("console").start()

22/05/24 11:35:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/y4/h7s1_2k959z8nfw5xqv11nmr0000gn/T/temporary-556df6fd-152f-4ec1-b583-6735b612a331. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/24 11:35:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|hocine|    1|
+------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|kafka|    1|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|spark|    1|
+-----+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|kafka|    2|
+-----+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|spark|    2|
+-----+-----+

